In [ ]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

In [ ]:
df = pd.read_csv('/final_data/data.csv')
df.pop('label')

columns = df.columns.values
rand = random.randint(0, len(columns))

features = df.values[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'komentar': '<username> Simulasi indonesia kembali dijajah keturunan belanda kedepan... bye bye talenta anak bangsa lokal indonesia... bubarkan liga dan pencarian bibit unggul atlet bola drpd rusak mental anak bangsanya...'}

In [ ]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, 'utf-8')]
        ),
    )

def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, str):
            feature_spec[keys] = string_feature(values)

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{
        'examples': {
            'b64': base64.b64encode(example).decode()
        }
    }]

    return json.dumps({
        'signature_name': 'serving_default',
        'instances': result,
    })

In [ ]:
def predict(input):
  response = requests.post(
      'https://analisis-sentimen-sty-production-ab80.up.railway.app/v1/models/sentimen-analysis-sty-model:predict',
      data=prepare_json(input)
  )

  pred = response.json()['predictions'][0][0]
  return pred

predict = predict(inputs)
predict
print(f'sentimen positif' if predict>0.5 else f'sentimen negatif')

sentimen negatif
